<a href="https://colab.research.google.com/github/JSJeong-me/AI-Innovation-2024/blob/main/Transformer/5-4-Zero_Shot_Audio_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 5: Zero-Shot Audio Classification

- In the classroom, the libraries have already been installed for you.
- If you are running this code on your own machine, please install the following:
```
    !pip install transformers
    !pip install datasets
    !pip install soundfile
    !pip install librosa
```

The `librosa` library may need to have [ffmpeg](https://www.ffmpeg.org/download.html) installed.
- This page on [librosa](https://pypi.org/project/librosa/) provides installation instructions for ffmpeg.

In [1]:
!pip install transformers
!pip install datasets
!pip install soundfile
!pip install librosa

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.6 MB/s eta 0:00:00


- Here is some code that suppresses warning messages.

In [2]:
# Hugging Face 토큰 설정 (필요 시 사용)
from google.colab import userdata
from huggingface_hub import login

# 비밀 정보에서 Hugging Face 토큰 가져오기
huggingface_token = userdata.get('HF_TOKEN')

In [3]:
from transformers.utils import logging
logging.set_verbosity_error()

### Prepare the dataset of audio recordings

In [4]:
from datasets import load_dataset, load_from_disk

# This dataset is a collection of different sounds of 5 seconds
dataset = load_dataset("ashraq/esc50",
                      split="train[0:10]")
# dataset = load_from_disk("./models/ashraq/esc50/train")

README.md:   0%|          | 0.00/345 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


dataset_infos.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

(…)-00000-of-00002-2f1ab7b824ec751f.parquet:   0%|          | 0.00/387M [00:00<?, ?B/s]

(…)-00001-of-00002-27425e5c1846b494.parquet:   0%|          | 0.00/387M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [14]:
audio_sample = dataset[0]

In [12]:
audio_sample

{'filename': '1-100210-A-36.wav',
 'fold': 1,
 'target': 36,
 'category': 'vacuum_cleaner',
 'esc10': False,
 'src_file': 100210,
 'take': 'A',
 'audio': {'path': None,
  'array': array([-0.00695801, -0.01251221, -0.01126099, ...,  0.215271  ,
         -0.00875854, -0.28903198]),
  'sampling_rate': 44100}}

In [13]:
from IPython.display import Audio as IPythonAudio
IPythonAudio(audio_sample["audio"]["array"],
             rate=audio_sample["audio"]["sampling_rate"])

### Build the `audio classification` pipeline using 🤗 Transformers Library

In [15]:
from transformers import pipeline

In [16]:
from datasets import load_dataset
from transformers import pipeline

# dataset = load_dataset("ashraq/esc50")
# audio = dataset["train"]["audio"][-1]["array"]

zero_shot_classifier = pipeline(task="zero-shot-audio-classification", model="laion/clap-htsat-unfused")
# output = audio_classifier(audio, candidate_labels=["Sound of a dog", "Sound of vaccum cleaner"])
# print(output)


config.json:   0%|          | 0.00/5.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/615M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

In [ ]:
# zero_shot_classifier = pipeline(
#     task="zero-shot-audio-classification",
#     model="./models/laion/clap-htsat-unfused")

More info on [laion/clap-htsat-unfused](https://huggingface.co/laion/clap-htsat-unfused).

### Sampling Rate for Transformer Models
- How long does 1 second of high resolution audio (192,000 Hz) appear to the Whisper model (which is trained to expect audio files at 16,000 Hz)?

In [17]:
(1 * 192000) / 16000

12.0

- The 1 second of high resolution audio appears to the model as if it is 12 seconds of audio.

- How about 5 seconds of audio?

In [18]:
(5 * 192000) / 16000

60.0

- 5 seconds of high resolution audio appears to the model as if it is 60 seconds of audio.

In [19]:
zero_shot_classifier.feature_extractor.sampling_rate

48000

In [20]:
audio_sample["audio"]["sampling_rate"]

44100

* Set the correct sampling rate for the input and the model.

In [21]:
from datasets import Audio

In [22]:
dataset = dataset.cast_column(
    "audio",
     Audio(sampling_rate=48_000))

In [36]:
audio_sample = dataset[1] # dataset['train'][0]

In [37]:
audio_sample

{'filename': '1-100038-A-14.wav',
 'fold': 1,
 'target': 14,
 'category': 'chirping_birds',
 'esc10': False,
 'src_file': 100038,
 'take': 'A',
 'audio': {'path': None,
  'array': array([-0.01288922, -0.09524129, -0.14230728, ...,  0.03312215,
          0.00153297,  0.        ]),
  'sampling_rate': 48000}}

In [38]:
candidate_labels = ["Sound of a dog",
                    "Sound of vacuum cleaner"]

In [39]:
zero_shot_classifier(audio_sample["audio"]["array"],
                     candidate_labels=candidate_labels)

[{'score': 0.8776861429214478, 'label': 'Sound of a dog'},
 {'score': 0.12231393158435822, 'label': 'Sound of vacuum cleaner'}]

In [40]:
candidate_labels = ["Sound of a child crying",
                    "Sound of vacuum cleaner",
                    "Sound of a bird singing",
                    "Sound of an airplane"]

In [41]:
zero_shot_classifier(audio_sample["audio"]["array"],
                     candidate_labels=candidate_labels)

[{'score': 0.9916227459907532, 'label': 'Sound of a bird singing'},
 {'score': 0.006114117801189423, 'label': 'Sound of an airplane'},
 {'score': 0.0015632470604032278, 'label': 'Sound of a child crying'},
 {'score': 0.0006998507888056338, 'label': 'Sound of vacuum cleaner'}]